# Is Child-Directed Speech Effective Training Data for Language Models?

## Предустановки

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install -q datasets transformers accelerate sentencepiece evaluate scipy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 28.1 MB/s eta 0:00:00


In [3]:
import os
import random
from datasets import load_dataset
import datasets

In [4]:
SEED = 42
random.seed(SEED)

In [5]:
!git clone https://github.com/styfeng/TinyDialogues.git
%cd TinyDialogues

Cloning into 'TinyDialogues'...
remote: Enumerating objects: 5730, done.
remote: Counting objects: 100% (946/946), done.
remote: Compressing objects: 100% (323/323), done.
remote: Total 5730 (delta 208), reused 920 (delta 191), pack-reused 4784 (from 3)
Receiving objects: 100% (5730/5730), 285.27 MiB | 50.21 MiB/s, done.
Resolving deltas: 100% (1283/1283), done.
Filtering content: 100% (4/4), 388.72 MiB | 92.14 MiB/s, done.
/content/TinyDialogues


## Датасет

In [6]:
# официальный датасет с huggingface
ds = load_dataset("styfeng/TinyDialogues")

print(ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

tinydialogue_train_ordered.txt:   0%|          | 0.00/141M [00:00<?, ?B/s]

tinydialogue_val_ordered.txt:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110024 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19708 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 110024
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 19708
    })
})


In [7]:
# =================================================
# Convert dataset to paper‑compatible text format
# =================================================
# Paper requirements (Appendix B):
# - One conversation per example
# - Speaker labels surrounded by ** **
# - Double newlines between utterances
# - <|endoftext|> token at end

os.makedirs("TD", exist_ok=True)

MAX_TOKENS = 5_000_000  # subset for Colab feasibility

def write_split(split, out_path):
    token_count = 0
    with open(out_path, "w", encoding="utf-8") as f:
        for ex in ds[split]:
            text = ex["text"].strip()
            if not text.endswith("<|endoftext|>"):
                text += "\n<|endoftext|>"
            tokens = text.split()
            if token_count + len(tokens) > MAX_TOKENS:
                break
            f.write(text + "\n")
            token_count += len(tokens)
    print(f"Wrote {token_count:,} tokens to {out_path}")

write_split("train", "TD/train.txt")
write_split("validation", "TD/val.txt")

Wrote 4,999,850 tokens to TD/train.txt
Wrote 4,382,089 tokens to TD/val.txt


In [8]:
print(ds["train"][0]["text"][:500])

**Dad**: "Hey sweetie, do you want to paint with Daddy?" \n\n **Child**: "Paint!" \n\n **Dad**: "Yes, we'll use these brushes. But first, let's put on your apron so we don't get paint on your clothes." \n\n **Child**: "Apron!" \n\n **Mom**: "Breakfast is almost ready! Who wants pancakes?" \n\n **Child**: "Pancake!" \n\n **Dad**: "We'll eat first, then paint. Let's wash hands before we eat, okay?" \n\n **Child**: "Wash!" \n\n **Mom**: "Careful, the pancakes are hot. We'll let them cool a little b


In [9]:
!python scripts/tokenizers/train_GPT2_tokenizer.py \
    TD/train.txt TD/val.txt TD_tokenizer

tokenizer_config.json: 100% 26.0/26.0 [00:00<00:00, 279kB/s]
config.json: 100% 665/665 [00:00<00:00, 7.97MB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 5.53MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 46.0MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 19.6MB/s]
['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']
50129
[00:00:00] Tokenize words                 ██████████████████ 46911    /    46911
[00:00:00] Count pairs                    ██████████████████ 46911    /    46911
[00:00:06] Compute merges                 ██████████████████ 51742    /    51742
All special tokens: ['<|endoftext|>', '<UNK>']
BOS token: <|endoftext|>
EOS token: <|endoftext|>
PAD token: None
UNK token: <|endoftext|>
SEP token: None
CLS token: None
MASK token: None


In [10]:
!python scripts/tokenizers/test_GPT2_tokenizer.py TD_tokenizer



 TD_tokenizer 

['do', 'Ġyou', 'Ġwant', 'Ġto', 'Ġlook', 'Ġat', 'Ġthat', 'Ġit', 'Ġsays', 'Ġlook', 'Ġ?']
do you want to look at that it says look ? 

['The', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġ(', "'", 'C', 'or', 'vin', 'ella', 'Ġcor', 'v', 'ina', "')", 'Ġis', 'Ġa', 'Ġlarge', 'Ġpasser', 'ine', 'Ġbird', 'Ġin', 'Ġthe', 'Ġshri', 'ke', 'Ġfamily', '.', 'ĠIt', 'Ġis', 'Ġsometimes', 'Ġknown', 'Ġas', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', 'Ġbut', 'Ġthis', 'Ġis', 'Ġto', 'Ġbe', 'Ġdiscouraged', ',', 'Ġsince', 'Ġit', 'Ġinvites', 'Ġconfusion', 'Ġwith', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', "Ġ'", 'L', 'an', 'ius', 'Ġsch', 'ach', "',", 'Ġof', 'Ġtropical', 'Ġsouthern', 'ĠAsia', '.', 'ĠThe', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġis', 'Ġa', 'Ġcommon', 'Ġresident', 'Ġbreeding', 'Ġbird', 'Ġin', 'Ġtropical', 'ĠAfrica', 'Ġfrom', 'ĠS', 'ene', 'gal', 'Ġeast', 'wards', 'Ġto', 'ĠU', 'g', 'anda', 'Ġand', 'Ġlocally', 'Ġin', 'Ġwesternmost', 'ĠKeny', 'a', '.', 'ĠIt', 'Ġfrequ',

Я посмотрела код этого скрипта с проверкой, там есть предложения, которые он токенизирует, и действительно всё правильно.
Можем сравнить с их токенайзером, там вроде всё так же, поэтому должно быть одинаково.


In [11]:
!python scripts/tokenizers/test_GPT2_tokenizer.py tokenizers/GPT2_tinydialogue



 tokenizers/GPT2_tinydialogue 

['do', 'Ġyou', 'Ġwant', 'Ġto', 'Ġlook', 'Ġat', 'Ġthat', 'Ġit', 'Ġsays', 'Ġlook', 'Ġ', '?']
do you want to look at that it says look ? 

['The', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġ(', "'", 'Cor', 'vin', 'ella', 'Ġcor', 'v', 'ina', "')", 'Ġis', 'Ġa', 'Ġlarge', 'Ġpasser', 'ine', 'Ġbird', 'Ġin', 'Ġthe', 'Ġshri', 'ke', 'Ġfamily', '.', 'ĠIt', 'Ġis', 'Ġsometimes', 'Ġknown', 'Ġas', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', 'Ġbut', 'Ġthis', 'Ġis', 'Ġto', 'Ġbe', 'Ġdiscouraged', ',', 'Ġsince', 'Ġit', 'Ġinvites', 'Ġconfusion', 'Ġwith', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', "Ġ'", 'L', 'an', 'ius', 'Ġsc', 'ha', 'ch', "',", 'Ġof', 'Ġtropical', 'Ġsouthern', 'ĠAsia', '.', 'ĠThe', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġis', 'Ġa', 'Ġcommon', 'Ġresident', 'Ġbreeding', 'Ġbird', 'Ġin', 'Ġtropical', 'ĠAfrica', 'Ġfrom', 'ĠSen', 'eg', 'al', 'Ġeast', 'ward', 's', 'Ġto', 'ĠU', 'g', 'anda', 'Ġand', 'Ġlocally', 'Ġin', 'Ġwesternmost', 'ĠK', 'eny'

Да, всё так.

## Обучаем GPT-2 на коленке

In [12]:
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer
from datasets import load_dataset

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [13]:
tokenizer = AutoTokenizer.from_pretrained("TD_tokenizer")
tokenizer.pad_token = tokenizer.eos_token

In [14]:
data_files = {
    "train": "TD/train.txt",
    "validation": "TD/val.txt"
}

raw_datasets = load_dataset("text", data_files=data_files)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [15]:
def sample_dataset(dataset, sample_fraction=0.3, seed=None):
    if seed is not None:
        random.seed(seed)

    sampled_dataset = {}
    for split_name in dataset:
        split = dataset[split_name]
        total_size = len(split)
        sample_size = int(total_size * sample_fraction)

        # Случайным образом выбираем индексы без повторов
        sampled_indices = random.sample(range(total_size), sample_size)

        # Создаём новый split с выбранными примерами
        sampled_dataset[split_name] = split.select(sampled_indices)

    return datasets.DatasetDict(sampled_dataset)

# Применяем выборку (30% от исходного датасета)
sampled_datasets = sample_dataset(raw_datasets, sample_fraction=0.3)

К сожалению нет времени полностью обучать, обучим хотя бы на 30% данных

In [16]:
sampled_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9126
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 5912
    })
})

In [17]:
def tokenize_fn(examples):
    out = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized_datasets = raw_datasets.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"],
)

Map:   0%|          | 0/30421 [00:00<?, ? examples/s]

Map:   0%|          | 0/19708 [00:00<?, ? examples/s]

In [18]:
# GPT‑2 SMALL config (124M params, as in paper)
config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions=1024,
    n_ctx=1024,
)

In [19]:
model = GPT2LMHeadModel(config)

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
training_args = TrainingArguments(
    output_dir="gpt2_td",
    overwrite_output_dir=True,
    # evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=5,  # paper uses 20; reduced for Colab
    weight_decay=0.0,
    logging_steps=200,
    save_strategy="epoch",
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-4026111171.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 0, 'bos_token_id': 0, 'pad_token_id': 0}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


In [ ]:
# =================================================
# Save model
# =================================================

trainer.save_model("/content/drive/My Drive/NLPproject/gpt2_td_final")
tokenizer.save_pretrained("/content/drive/My Drive/NLPproject/gpt2_td_final")

print("Training complete. Model saved.")

# =================================================
# NEXT STEPS (not run here):
# -------------------------------------------------
# - Zorro evaluation (BabyLM pipeline)
# - Word Similarity benchmarks
# - Dataset comparisons (Wikipedia, CHILDES)
# - Local / global ordering experiments
# =================================================


Training complete. Model saved.
